# Week 3 project - segmenting and clustering neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

#### Scrape Wikipedia for Toronto data using Beautiful Soup package

https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

#### Sort of helpful Youtube tutorial https://www.youtube.com/watch?v=ng2o98k983k

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import xml
import pandas as pd

#### Gets the HTML of the entire Wikipedia page as a string

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Apply Beautiful Soup to make Soup using URL, then apply Prettify to view nested tags

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml') # make Soup using URL

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

#### The table is located under <"table class="wikitable sortable">

The HTML script of all the table contents is under the class wikitable sortable, where there are links to the neighborhood and postal code

Access tags by treating them as attributes when using soup.find()
First locate the table, then parce out each neighborhood using 'tr'

In [4]:
neighborhood = soup.find('table', class_ = 'wikitable')
neighborhood_rows = neighborhood.find_all('tr')

#### Extract the columns 'Postcode', 'Borough', 'Neighbourhood' from the table


In [5]:
information = []
for row in neighborhood_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (the first and last items)
    information.append(info)
    
information[0:20] #preview the first 20 rows

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District']]

#### Turn above information table into a Pandas dataframe

In [6]:
neighbor_df = pd.DataFrame(information[1:], columns=information[0])
# where information[1:] contains each row of neighborhoods
# and columns = information[0] gives the column names


# Re-spell Neighbourhood as Neighborhood
neighbor_df = neighbor_df.rename(columns={neighbor_df.columns[2]: "Neighborhood" })


neighbor_df.head(20)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Data cleansing of the neighbor_df

1. Get rid of Boroughs with "Not assigned"
2. Combine Neighborhoods that share the same postcode
3. For any Neighborhoods that are "not assigned," change to the Borough

#### Dropping any row where Borough is not assigned

In [7]:
neighbor_df = neighbor_df[neighbor_df.Borough != 'Not assigned']

neighbor_df.reset_index(drop=True, inplace=True)

neighbor_df.head(20)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Combine neighborhoods with the same postcode

Helpful: https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings

In [8]:
grouped = neighbor_df.groupby(['Postcode']) # group by Postcode


# combine the neighborhoods grouped by postcode and into a new df
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: x.sum()) 
# adds spaces and commas between neighborhoods
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
# matches a borough to each postcode
borough_grouped = grouped['Borough'].apply(lambda x: set(x).pop())
# turn borough_grouped and neighborhood_grouped into dataframes
borough = borough_grouped.to_frame()
neighborhood = neighborhood_grouped.to_frame()
#combine the dataframe borough and the dataframe neighborhood into one dataframe
grouped_final = borough.merge(neighborhood, on="Postcode")

grouped_final

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


#### Print the .shape method to print the number of rows of the dataframe

In [9]:
print('The number of rows and columns in this final grouped dataframe is',grouped_final.shape) 

The number of rows and columns in this final grouped dataframe is (103, 2)






## Attach longitude and latitude to the postcode, borough, and neighborhood table

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

Given that the Geocoder Python package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Change the column name of Postal Code to Postcode

In [11]:
geospatial_data = geospatial_data.rename(columns={geospatial_data.columns[0]: "Postcode" })


In [12]:
full_table = grouped_final.merge(geospatial_data, on = 'Postcode')

full_table.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore and cluster the neighborhoods of Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Make sure to:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together. 

#### First, install some necessary packages

In [13]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: / 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-h1de35cc_0
  - anaconda::certifi-2018.11.29-py37_0, anaconda::openssl-1.1.1a-h1de35cc_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.1.1a-h1de35cc_0, defaults::certifi-2018.11.29-py37_0
  - anaconda::openssl-1.1.1a-h1de35cc_0, defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0
  - anaconda::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-h1de35cc_0
  - defaults::ca-certificates-2018.03.07-0, defaults::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-h1de35cc_0
  - anaconda::ca-certificates-2018.03.07-0, anaconda::certifi-2018.11.29-py37_0, defaults::openssl-1.1.1a-h1de35cc_0
  - anaconda::certifi-2018.11.29-py37_0, defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.1.1a-h1de35ccdone

# All requested packages already inst

In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


#### Let's see how many boroughs and neighborhoods are in Toronto

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(full_table['Borough'].unique()),
        full_table['Neighborhood'].shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Get the longitude/latitude of Toronto using geolocator

In [16]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [17]:
# create a map of Toronto
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor, neigh in zip(full_table['Latitude'], full_table['Longitude'], 
                                 full_table['Borough'], full_table['Neighborhood']):
    label = '{}, {}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
        
map_toronto

The map above shows all 103 neighborhoods in Toronto, using Folium map and a circle marker in the colors of the Canadian flag. Hovering over the markers, a label shows the neighborhood and the borough names.

#### Next, let's cluster by borough by first getting the geographic coordinates of each borough, then visualize via Folium

In [18]:
list_boroughs = full_table['Borough'].unique()
list_boroughs

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [20]:
def borough_loc(list_of_places):
    for place in list_of_places:
        address = (place + ", Ontario, Canada")
        geolocator = Nominatim(user_agent="TO_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{''}, {}, {},'.format(place,latitude,longitude))

borough_loc(list_boroughs)

Scarborough, 43.773077, -79.257774,
North York, 43.7708175, -79.4132998,
East York, 43.6913391, -79.3278212,
East Toronto, 43.653963, -79.387207,
Central Toronto, 43.653963, -79.387207,
Downtown Toronto, 43.655115, -79.380219,
York, 44.0007518, -79.4372217,
West Toronto, 43.653963, -79.387207,
Queen's Park, 43.6599803, -79.3903686,
Mississauga, 43.590338, -79.645729,
Etobicoke, 43.6435559, -79.5656326,


In [21]:
import numpy as np

boroughs = ['Scarborough', 43.773077, -79.257774,
'North York', 43.7708175, -79.4132998,
'East York', 43.6913391, -79.3278212,
'East Toronto', 43.653963, -79.387207,
'Central Toronto', 43.653963, -79.387207,
'Downtown Toronto', 43.655115, -79.380219,
'York', 44.0007518, -79.4372217,
'West Toronto', 43.653963, -79.387207,
"Queen's Park", 43.6599803, -79.3903686,
'Mississauga', 43.590338, -79.645729,
'Etobicoke', 43.6435559, -79.5656326]

boroughs_df = pd.DataFrame(np.array(boroughs).reshape(11,3), columns = ["Borough","Latitude","Longitude"])

boroughs_df

,Borough,Latitude,Longitude
0,Scarborough,43.773077,-79.257774
1,North York,43.7708175,-79.4132998
2,East York,43.6913391,-79.3278212
3,East Toronto,43.653963,-79.387207
4,Central Toronto,43.653963,-79.387207
5,Downtown Toronto,43.655115,-79.380219
6,York,44.0007518,-79.4372217
7,West Toronto,43.653963,-79.387207
8,Queen's Park,43.6599803,-79.3903686
9,Mississauga,43.590338,-79.645729


In [22]:
boroughs_df.dtypes

boroughs_df["Latitude"] = pd.to_numeric(boroughs_df["Latitude"])
boroughs_df["Longitude"] = pd.to_numeric(boroughs_df["Longitude"])


In [23]:
# create a map of Toronto
map_toronto_boroughs = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], 
                                 boroughs_df['Borough']):
    label = '{}'.format(bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto_boroughs)
        
map_toronto_boroughs

#### I want to explore the various neighborhoods within the boroughs. I want to first see which borough has the most neighborhoods, then focus on a particularly populated borough.

In [24]:
full_table['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

It looks like North York has the most neighborhoods, with 24, followed by Downtown Toronto and Scarborough. I'm curious about the urban part of Toronto, so let's explore Downtown Toronto.

In [25]:
# The geographical coordinates for Downtown Toronto
DT_lat = boroughs_df.iloc[5,1]
DT_long = boroughs_df.iloc[5,2]

# The dataframe that contains all the Downtown Toronto neighborhoods
DT_df = full_table[full_table['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

# create map of Downtown Toronto neighborhoods using latitude and longitude values
map_DT = folium.Map(location=[DT_lat, DT_long], zoom_start=12.4)

# add markers to map
for lat, lng, label in zip(DT_df['Latitude'], DT_df['Longitude'], DT_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

#### Let's play with some Foursquare data!

In [26]:
# Define Foursquare Credentials and Versions

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20181102' # Foursquare API version


#### Let's explore the first neighborhood in our dataframe.

Get the first neighborhood's name

In [27]:
DT_df.loc[0, 'Neighborhood']

'Rosedale'

Get the Rosedale's longitude and latitude

In [28]:
neighborhood_latitude = DT_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DT_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


#### Now I will get the top 100 venues in Rosedale within a 500 meter radius.

First, I will create the GET request

In [29]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat, 
    lng, 
    VERSION, 
    radius, 
    LIMIT)

#### Send GET request and examine output

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c632f264434b95ecf3c678a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Christie Pits',
  'headerFullLocation': 'Christie Pits, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.6740420045, 'lng': -79.41635411972038},
   'sw': {'lat': 43.6650419955, 'lng': -79.42877328027961}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcfd7cf964a5203e6321e3',
       'name': 'Fiesta Farms',
       'location': {'address': '200 Christie St',
        'crossStreet': 'at Essex St',
        'lat': 43.66847077052224,
        'lng': -79.42048512748114,
        'labeledLatLngs': [{'label': 'd

#### Here is a function to get the venues into a dataframe form

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Starbucks,Coffee Shop,43.671585,-79.421366
3,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
4,Universal Grill,Diner,43.670550,-79.426541
5,Scout and Cash Caffe,Café,43.667360,-79.419938
6,Loblaws,Grocery Store,43.671807,-79.421102
7,Actinolite,Restaurant,43.667858,-79.428054
8,Faema Caffe,Café,43.672003,-79.422209
9,Marlenes Just Babies,Baby Store,43.671824,-79.420499


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


#### Next, here's a function to get the venues for all the neighborhoods in Downtown Toronto

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                lat, 
                lng, 
                VERSION, 
                radius, 
                LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *DT_venues*.

In [35]:
DT_venues = getNearbyVenues(names=DT_df['Neighborhood'],
                                   latitudes=DT_df['Latitude'],
                                   longitudes=DT_df['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


#### Here's the size and preview of the resulting dataframe

In [36]:
print(DT_venues.shape)
DT_venues.head()

(1280, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Here are the venues for each neighborhood

In [37]:
DT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [38]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))

There are 207 uniques categories.


#### Analyze each neighborhood's venues

In [39]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [40]:
DT_onehot.shape

(1280, 207)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.03,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1    Restaurant  0.05
2  Cocktail Bar  0.05
3        Bakery  0.04
4      Beer Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.09
1         Park  0.07
2   Restaurant  0.07
3  Pizza Place  0.04
4          Pub  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restaurant  0.05
3                 Bar  0.04
4        Burger Joint  0.04


---

#### Some insights from the top venue per neighborhood analysis:

No surprise that in the Island Airport, Railway Lands neighborhoods, the top venues are airport lounges, airport service, airport terminal, plane, boat or ferry. These neighborhoods seem to be mostly have commuter related venues.

The top venues in Chinatown and Kensington Market are food establishments: Vietnamese and vegan restaurants, bars, bakeries, and cafes.

Church and Wellesley are neighborhoods that also have lots of food options, specifically Japanese cuisine.

Christie is the neighborhood to go to for eating in, which a high concentration of grocery and convenience stores. Eating in means you have more money to spend on nightclubs!

Craving caffeine and an afternoon snack? Central Bay Street is the neighborhood for you! There's many coffee shops and cafes and even a few burger joints.


#### Put the top venues information into a dataframe

First, write a function to sort the venues in descending order

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Clothing Store,Asian Restaurant,Restaurant,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Steakhouse,Cheese Shop,Italian Restaurant,Beer Bar,Seafood Restaurant,Bakery
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Pub,Bakery,Café,Italian Restaurant,Japanese Restaurant,Bank
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Salad Place,Ice Cream Shop,Indian Restaurant,Spa,Thai Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Gaming Cafe
6,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Athletics & Sports,Diner,Baby Store,Nightclub,Convenience Store
7,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Bubble Tea Shop,Café,Men's Store,Mediterranean Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Steakhouse,Italian Restaurant,Seafood Restaurant,Deli / Bodega
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Seafood Restaurant


Either Foursquare just has a lot of coffee shop data, or Downtown Torontonians really love their coffee!


#### Cluster the Neighborhoods of Downtown Toronto

In [45]:
# run K-means to cluster the neighborhoods into 5 clusters

# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 3, 3, 0, 4, 0, 0, 3], dtype=int32)

#### Create a new dataframe that inclues the cluster as well as the top 10 venues

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

DT_merge = DT_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merge = DT_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merge.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,3,Coffee Shop,Restaurant,Park,Pizza Place,Pub,Bakery,Café,Italian Restaurant,Japanese Restaurant,Bank
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Bubble Tea Shop,Café,Men's Store,Mediterranean Restaurant
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Tea Room,Theater,Pizza Place,Thai Restaurant,Diner


#### Examine Clusters

Cluster 1

In [53]:
DT_merge.loc[DT_merge['Cluster'] == 0, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Church and Wellesley,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Bubble Tea Shop,Café,Men's Store,Mediterranean Restaurant
4,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Tea Room,Theater,Pizza Place,Thai Restaurant,Diner
5,St. James Town,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Italian Restaurant,Bakery,Breakfast Spot,Park,Gastropub
6,Berczy Park,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Steakhouse,Cheese Shop,Italian Restaurant,Beer Bar,Seafood Restaurant,Bakery
8,"Adelaide, King, Richmond",0,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Clothing Store,Asian Restaurant,Restaurant,Gym
11,"Commerce Court, Victoria Hotel",0,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Steakhouse,Italian Restaurant,Seafood Restaurant,Deli / Bodega
12,"Harbord, University of Toronto",0,Café,Bakery,Japanese Restaurant,Bar,Coffee Shop,Bookstore,Restaurant,Pub,Jazz Club,Beer Bar
13,"Chinatown, Grange Park, Kensington Market",0,Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Gaming Cafe
15,Stn A PO Boxes 25 The Esplanade,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Pub,Beer Bar,Cocktail Bar,Seafood Restaurant,Hotel,Cheese Shop
16,"First Canadian Place, Underground city",0,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Deli / Bodega,Bar,Asian Restaurant


Cluster 2

In [54]:
DT_merge.loc[DT_merge['Cluster'] == 1, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,1,Park,Playground,Trail,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 3

In [55]:
DT_merge.loc[DT_merge['Cluster'] == 2, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",2,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden


Cluster 4

In [56]:
DT_merge.loc[DT_merge['Cluster'] == 3, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",3,Coffee Shop,Restaurant,Park,Pizza Place,Pub,Bakery,Café,Italian Restaurant,Japanese Restaurant,Bank
3,"Harbourfront, Regent Park",3,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
7,Central Bay Street,3,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Salad Place,Ice Cream Shop,Indian Restaurant,Spa,Thai Restaurant
9,"Harbourfront East, Toronto Islands, Union Station",3,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery
10,"Design Exchange, Toronto Dominion Centre",3,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Seafood Restaurant


Cluster 5

In [57]:
DT_merge.loc[DT_merge['Cluster'] == 4, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,4,Grocery Store,Café,Park,Coffee Shop,Restaurant,Athletics & Sports,Diner,Baby Store,Nightclub,Convenience Store
